В данной статье описан интересный проход к некоторым image-to-image задачам, таким как уменьшение шума на изображении (denoising), увеличение разрешения (super-resolution) и дорисовывание недостающих участков на изображении (inpainting). При этом *не используется никакого обучающего датасета*, а используется лишь то изображение, над которым нужно выполнить преобразование. Интересен не только метод сам по себе, но и выводы, которые из него можно сделать.

### Blind denoising

Задачей blind denoising называется задача уменьшения шума на изображении $x_{noisy}$, при этом вероятностное распределение шума заранее неизвестно. Это может быть, например, наложенный белый шум или артефакты jpeg.

Создадим случайно инициализированную сверточную нейронную сеть $f(z, \theta)$, которая преобразует тензор признаков $z$ в изображение, используя веса $\theta$. Например, можно взять сеть [U-Net]($U-Net: Convolutional Networks for Biomedical Image Segmentation$), если интерпретировать ее входные данные не как изображение, а как тензор признаков. В качестве $z$ будем использовать массив случайных чисел. Будем решать следующую задачу оптимизации (например, градиентным спуском):

$\|f(z, \theta) - x_{noisy}\|^2 \to \underset{\theta}{\min}$

То есть мы оптимизируем веса сети так, чтобы случайный шум она преобразовывала в исходное изображение $x_{noisy}$. Если долго и упорно оптимизировать $\theta$, то как правило $f(z, \theta)$ станет почти точной копией $x_{noisy}$, особенно если в сети очень много параметров. Но если останавливать процесс оптимизации "на полпути" (early stopping), то $f(z, \theta)$ станет приблизительной копией $x_{noisy}$, избавленной от шума. То есть модель успевает обучиться воссоздавать детали изображения, но не успевает обучиться воссоздавать мелкодисперсный шум.

Авторы отмечают, что оптимизацию можно производить не только по весам сети $\theta$, но и по тензору признаков $z$, хотя они так не делают. Можно предположить, что оптимизация по $\theta$ и $z$ будет почти эквивалентна оптимизации более глубокой сети только по $\theta$ (в этом случае $z$ сам генерируется сверточной сетью).

> ...while it is also possible to optimize over the code $z$, in our experiments we do not do so.

Каждый шаг оптимизации можно искажать $z$ небольшим случайным шумом (noise-based regularization).

Интересно, что процесс оптимизации сходится достаточно быстро, если $x_{noisy}$ является настоящим изображением (рисунком или фотографией), и намного медленнее, если в качестве $x_{noisy}$ взять случайный шум. На иллюстрации ниже показаны графики функции потерь для исходных изображений (Image), изображений с наложенным шумом (Image + noise), изображений с переставленными пикселями (Image shuffled) и случайного шума ($U(0, 1)$ noise).

> Naturally-looking images result in much faster convergence, whereas noise is rejected.

<img src="assets/deepimageprior.jpg" width="1000" align="center">

Здесь, правда остается, вопрос: связана ли быстрая сходимость с тем, что изображение выглядит натурально? Вполне возможно, что быстрая сходимость обусловлена лишь близкими значениями находящихся рядом пикселей.

### Super-resolution

В задаче повышения разрешения изображений используется аналогичный подход: увеличенное изображение мы получаем как $x_{upscaled} = f(z, \theta)$. Чтобы сравнить $x_{upscaled}$ с исходным $x_{orig}$, изображение $x_{upscaled}$ уменьшается до исходного размера каким-либо дифференцируемым алгоритмом, например Lanczos:

$\|\textit{LanczosDownscale}(f(z, \theta)) - x_{orig}\|^2 \to \underset{\theta}{\min}$

В сравнении с bicubic upsampling, предложенный метод обеспечивает большую резкость границ на увеличенном изображении. Качество upsmpling'а приближается к качеству, достигаемому нейронными сетями, которые обучаются на больших датасетах.

На изображении ниже сравниваются разные способы восстановить фотографию зебры, предварительно уменьшенную до малого размера.

<img src="assets/deepimageprior3.jpg" width="1000" align="center">

"No prior" означает непосредственную оптимизацию среднеквадратичного отклонения без использования сверточной сети:

$\|\textit{LanczosDownscale}(z) - x_{orig}\|^2 \to \underset{\theta}{\min}$.

"TV prior" означает использование *total variation*, описанного в [другой работе]($Understanding Deep Image Representations by Inverting Them$).

### Inpainting

В этой задаче исходными данными является изображение $x_{orig}$ и бинарная маска $x_{mask}$. Требуется реалистично дорисовать те участки изображения, где $x_{mask} = 0$. Такую задачу решает, например, инструмент "заполнение с учетом содержимого" в Photoshop.

Чтобы применить описанный метод к задаче inpainting, достаточно минимизировать среднеквадратичное отклонение по тем пикселям, где $x_{mask} = 1$. 

$\|(f(z, \theta) - x_{orig}) \odot x_{mask}\|^2 \to \underset{\theta}{\min}$

То есть сверточная сеть генерирует изображение, которое должно совпасть с исходным везде, кроме замаскированной области. При этом сгенерированный участок, который соответствует замаскированной области, мы считаем результатом inpainting'а.

Выбор learning rate при этом играет определяющую роль:

<img src="assets/deepimageprior4.jpg" width="1000" align="center">

В этой задаче особенно хорошо работают сети типа encoder-decoder, добавление skip connections ухудшает результат:

<img src="assets/deepimageprior5.jpg" width="800" align="center">

В случае, когда требуется дорисовать большие участки изображения, результат получается лучше, если в качестве $z$ взять не случайный шум, а заливку градиентом. В других задачах заливка градиентом не дает преимущества по сравнению со случайным шумом.

Вполне очевидно, что случайно инициализированная нейронная сеть не сможет дорисовать такие детали, которые больше нигде не встречались в исходном изображении. Например, не получится дорисовать голову кота, закрытую маской, поскольку в нейронную сеть не заложена информация о том, как выглядит кот, и эту информацию взять неоткуда. При данном подходе маска будет заполнена такими текстурами, которые присутствуют где-то еще вокруг маски.

### Обобщение

В общем случае предложенный метод авторы описывают как "минимизацию энергии" между сгенерированным и исходным изображением:

$E(f(z, \theta), x_{orig}) \to \underset{\theta}{\min}$

Функция $E$ зависит от задачи, например, в задаче шумоподавления $E(x, y) = \|x - y\|^2$.

Введение понятия "deep image prior" авторы поясняют следующим образом:

> In this work, we ... use the implicit prior captured by the neural network parametrization, as follows:
>
> $\theta^* = \underset{\theta}{\text{argmin}} E(f_\theta(z); x_0),\ \ \ \ \ \ \ \ \ x^* = f_{\theta^*}(z) \tag{2}$
>
> Since no aspect of the network $f_\theta$ is learned from data beforehand, such deep image prior is effectively handcrafted.
>
> The prior defined by eq. $(2)$ is implicit and does not define a proper probability distribution in the image space. Nevertheless, it is possible to draw "samples" (in the loose sense) from this prior by taking random values of the parameters $\theta$ and looking at the generated image $f_\theta(z)$. In other words, we can visualize the starting points of the optimization process eq. (2) before fitting the parameters to the noisy image. Figure 5 shows such  “samples” from the deep priors captured by different hourglass-type architectures.
>
> <img src="assets/deepimageprior2.jpg" width="800" align="center">
>
> Fig. 5: "Samples" from the deep image prior. We show images that are produced by ConvNets with random weights from independent random uniform noise. The scale of structures naturally changes with the depth of the network.
>
> Adding skip connections results in images that contain structures of different characteristic scales, as is desirable for modeling natural images. It is therefore natural that such architectures are the most popular choice for generative ConvNets.

В байесовском выводе под понятием prior понимается пространство гипотез, в котором осуществляется поиск, и то, какие гипотезы из этого пространства мы предпочитаем в большей или меньшей степени.

Например, в задаче inpainting, имея $x_{orig}$, мы *ищем ближайший элемент в пространстве $X$ всех изображений, сгенерированных сверточной сетью*:

$X = \{f(z, \theta)\ |\ \theta \in \Theta, z \in Z\}$

(строго говоря, это будет так, если мы будем выполнять оптимизацию также и по $z$, авторы упоминают о такой возможности)

Пространство $X$ авторы и называют deep image prior. Говоря более точно - это те элементы пространства $X$, к которым легче сходится процесс оптимизации. Поиск ближайшего элемента в $X$ можно применить в различных задачах, связанных с исправлением "повреждений" в изображении:

> Our approach does not require a model for the image degradation process that it needs to revert. This allows it to be applied in a “plug-and-play” fashion to image restoration tasks, where the degradation process is complex and/or unknown and where obtaining realistic data for supervised training is difficult.

Как мне кажется, в данном подходе все же есть некоторые проблемы. Во-первых, вероятно в $X$ содержатся практически все возможные изображения, так как количество параметров в сверточных сетях обычно во много раз превышает количество пикселей в изображении, и в результате система уравнений $x = f(\theta, z)$, решаемая относительно $\theta$, становится недоопределенной и имеет много решений. Это подтверждается тем фактом, что в задаче шумоподавления длительная оптимизация $\theta$ приводит к исходному, зашумленному изображению, из-за чего необходима ранняя остановка. Но с другой стороны семплирование из $X$ (см. выше) приводит к изображениям, непохожим на случайным шум.

Также в $X$ содержатся всевозможные размытые изображения, поэтому маловероятно, что задачу обращения размытия (deblurring) возможно решить таким методом. И наконец, deep image prior никак не позволяет дополнить изображение сложными деталями исходя из его семантики, например дорисовать руку человека, закрытую маской. Но это, конечно, не уменьшает значимости данной работы.

Еще одной проблемой, конечно, является скорость работы: для каждого изображения требуется выполнять процесс оптимизации.

### Наличие обучающих данных

Авторы утверждают, что в их методе вообще не происходит обучения:

> In this work, we show that, in fact, not all image priors must be learned from data; instead, a great deal of image statistics are captured by the structure of generator ConvNets, independent of learning. This is especially true for the statistics required to solve certain image restoration problems, where the image prior must supplement the information lost in the degradation processes.
> 
> ...no aspect of the network is learned from data and illustrates the power of the image prior implicitly captured by the network structure. To the best of our knowledge, this is the first study that directly investigates the prior captured by deep convolutional generative networks independently of learning the network parameters from images.

Предложенный метод интересен своей простотой и эффективностью, но сделанный вывод звучит несколько странно. Кажется необоснованным утверждение о том, что метод не использует обучающие данные. В каком-то смысле обучающими данными можно считать само изображение, на котором проводится оптимизация. Сеть обучается рисовать текстуры на участках, не закрытых маской, и дорисовывает такие же текстуры на участках, закрытых маской.

Особенно это будет видно, если inpainting будет проводиться на изображении очень большого размера (намного большего, чем рецептивное поле сети), например, $10^5 \times 10^5$ пикселей, некоторые небольшие части которого закрыты маской. По количеству пикселей это изображение будет сопоставимо с небольшим обучающим датасетом. Если не учитывать перекрытие рецептивных полей, то можно разрезать изображение на части, и задача станет эквивалентна обучению одной и той же сети дорисовывать сразу несколько изображений $x_i$, каждое по своему $z_i$. Сети придется выучивать много разных типов текстур, чтобы дорисовывать недостающие участки. Задача станет проще, если вместе с $\theta$ мы будем оптимизировать и $z_i$ (что авторы работы не запрещают).

В такой формулировке *алгоритм станет эквивалентным описанному в работе [Deconvolutional Networks]($Deconvolutional Networks$)* (2010). Отличие лишь в цели: в данной работе целью является обработка исходного изображения (дорисовывание, шумоподавление и т. д.) а в работе Deconvolutional Networks целью было нахождение карты признаков для последующей классификации.

### Инверсия промежуточных слоев нейронных сетей

Одной из возможностей интерпретации сверточных сетей является метод pre-image. Пусть мы хотим проанализировать промежуточные представления, возникающие на одном из слоев сверточной сети. Обозначим предыдущие слои сети как $\Phi$. Имея натуральное изображение $x_0$, мы можем получить его карту признаков $\Phi(x_0)$. Чтобы понять, какая информация заложена в $\Phi(x_0)$, мы можем найти другие изображения, которые дают такую же карту признаков.

Самым простым способом является инициализация $x_1$ случайным шумом или другим натуральным изображением и последующая оптимизация $x_1$ для минимизации различия между картами признаков для $x_0$ и $x_1$:

$\|\Phi(x_0) - \Phi(x_1)\|^2 \to \underset{x_1}{\min}$

Однако в этом случае оптимизация может привести к ненатурально выглядящему изображению (например, см. [Deep Dream](https://www3.cs.stonybrook.edu/~cse352/T12talk.pdf)). Поскольку сверточная сеть обучалась работе только с натуральными изображениями, ее поведение на ненатуральных изображениях является неопределенным и может быть каким угодно. Для получения более натуральных изображений можно применять различные регуляризаторы, ограничивающие пространство допустимых значений для $x_1$, то есть задающие априорное распределение для $x_1$, например, total variance prior ([Mahendran et al., 2014]($Understanding Deep Image Representations by Inverting Them$)).

Авторы предлагают использовать deep image prior в качестве такого априорного распределения. Таким образом мы решаем следующую задачу оптимизации:

$\|\Phi(x_0) - \Phi(\text{ConvNet}(z, \theta))\|^2 \to \underset{\theta}{\min}$

Такой подход имеет большое сходство с perceptual loss ([Johnson et al., 2016]($Perceptual Losses for Real-Time Style Transfer and Super-Resolution$)). На иллюстрации ниже сравниваются результаты инверсии [AlexNet]($ImageNet Classification with Deep Convolutional Neural Networks$), полученные тремя способами: deep image prior, total variance prior, а также нейронной сетью, обученной инвертировать карты признаков ([Dosovitskiy et al., 2015]($Inverting Visual Representations with Convolutional Networks$)).

<img src="assets/deepimageprior6.jpg" width="1000" align="center">

Еще один способ интерпретации сверточных сетей заключается в том, что мы ищем изображение, которое максимизирует один из элементов в карте признаков (см. [Visualizing and Understanding Convolutional Networks]($Visualizing and Understanding Convolutional Networks$)), либо значение на выходном нейроне, соответствующем одному из классов. В данном случае также можно ограничить пространство поиска для изображения с помощью deep image prior.

Ниже показаны результаты поиска изображений, максимизирующих logit'ы классов Black Swan, Goose, Frog, Cheeseburger. Эксперимент проводился на AlexNet и [VGG-16]($Very Deep Convolutional Networks for Large-Scale Image Recognition$) с использованием total variance prior и deep image prior. Как можно видеть, deep image prior дает более натуральные изображения, особенно в случае AlexNet.

<img src="assets/deepimageprior7.jpg" width="700" align="center">

### Резюме

Во многих задачах требуется выполнять поиск реалистичного изображения, обладающего некими заданными свойствами. Например, в задаче super-resolution мы ищем реалистичное изображение (без ряби и артефактов), уменьшение которого приводит к исходному изображению. В задаче шумоподавления мы также ищем реалистичное изображение, похожее на исходное, но обладающее меньшим шумом.

Авторы предлагают ограничивать пространство поиска только теми изображениями, которые могут быть представлены как результат обработки некой карты признаков $z$ сверточной сетью $f$ (формула 1; в качестве $f$ авторы используют U-Net). Это пространство поиска $X$ и называется deep image prior.

**Пример 1.** Denoising. Имея зашумленное изображение $x_{noisy}$, мы ищем ближайший к $x_{noisy}$ элемент в пространстве $X$ (формула 2). Задача решается градиентным спуском, при этом мы используем early stopping, чтобы сеть успела воспроизвести детали изображения, но не успела обучиться воспроизводить шум на нем (который мы и хотим удалить).

**Пример 2.** Super-resolution. Мы ищем элемент в пространстве $X$, downscaling которого приведет к исходному изображению (формула 3).

**Пример 3.** Inpainting. Мы ищем элемент в пространстве $X$, участки которого, не закрытые маской, оказываются как можно ближе к исходному изображению (формула 4).

<img src="assets/deepimageprior8.jpg" width="500" align="center">

В таком подходе не используется никаких обучающих данных кроме того изображения, над которым выполняется преобразование. Метод deep image prior чаще всего несколько уступает сетям, обученным на больших датасетах, но превосходит по качеству методы, работающие без обучения (например, bicubic upscaling для super-resolution). Также метод deep image prior дает качественные результаты в задаче интерпретации промежуточных активаций сверточных сетей (где нужно найти исходное изображение, имея его карту признаков). Таким образом, получается простой "швейцарский нож", позволяющий одним и тем же простым способом решать самые разные задачи. С другой стороны, этот метод требует оптимизации на каждом изображении, из-за чего работает медленно. В заключении авторы пишут:

> While of limited practicality, the good results of our approach across a wide variety of tasks demonstrate that an implicit prior inside deep convolutional network architectures is an important part of the success of such architectures for image restoration tasks.
>
>...properly hand-crafted network architectures correspond to better hand-crafted priors, and it seems that learning ConvNets builds on this basis. This observation also validates the importance of developing new deep learning architectures.